<a href="https://colab.research.google.com/github/jordan-dsouza/SlashMarkInternship/blob/main/content_based_music_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries



In [1]:
import numpy as np
import pandas as pd

In [2]:
from typing import List, Dict

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Dataset

In [12]:
songs = pd.read_csv("songdata.csv")

In [13]:
songs.head()

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


As dataset has large size, only 5000 songs resampled.

In [14]:
songs = songs.sample(n=5000).drop("link", axis=1).reset_index(drop=True)

Removing '\n'.

In [15]:
songs["text"] = songs["text"].str.replace(r"\n", "")

<ipython-input-15-3ab58cbcb722>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  songs["text"] = songs["text"].str.replace(r"\n", "")


TF-IDF vectorizer calculates the TF-IDF score for each song lyric word-by-word.

In [16]:
tfidf = TfidfVectorizer(analyzer="word", stop_words="english")

In [17]:
lyrics_matrix = tfidf.fit_transform(songs["text"])

Cosine similarity calculates the similarity of one lyric to another.


In [18]:
cosine_similarities = cosine_similarity(lyrics_matrix)

Once we get the similarities, we'll store in a dictionary the names of the 50  most similar songs for each song in our dataset.

In [19]:
similarities = {}

In [20]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # After that, we'll store in similarities each name of the 50 most similar songs.
    # Except the first one that is the same song.
    similarities[songs["song"].iloc[i]] = [(cosine_similarities[i][x], songs["song"][x], songs["artist"][x]) for x in similar_indices][1:]

Use similarity scores to access the most similar items and give a recommendation.

In [21]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [22]:
recommedations = ContentBasedRecommender(similarities)

##Recommendations

In [23]:
recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [24]:
recommedations.recommend(recommendation)

The 4 recommended songs for The Little Drummer Boy are:
Number 1:
One More Story by Peter Cetera with 0.267 similarity score
--------------------
Number 2:
Close Your Eyes by Harry Belafonte with 0.24 similarity score
--------------------
Number 3:
Good Night Sweetheart by Bing Crosby with 0.186 similarity score
--------------------
Number 4:
December Child by Cyndi Lauper with 0.174 similarity score
--------------------


And we can pick another random song and recommend again:

In [25]:
recommendation2 = {
    "song": songs['song'].iloc[120],
    "number_songs": 4
}

In [27]:
recommedations.recommend(recommendation2)

The 4 recommended songs for Custer are:
Number 1:
Anymore by Kelly Clarkson with 0.334 similarity score
--------------------
Number 2:
Paperclips by Travis with 0.28 similarity score
--------------------
Number 3:
Ride To Agadir by Boney M. with 0.246 similarity score
--------------------
Number 4:
It's Over by Ne-Yo with 0.217 similarity score
--------------------
